In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import implicit
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

# 1. load data

In [3]:
buy = pd.read_csv('../data/buy_data.csv') # 구매이력
history = pd.read_csv('../data/history_data.csv') # 시청 시작 데이터
watch = pd.read_csv('../data/watch_e_data.csv') # 시청 종료 데이터
meta = pd.read_csv('../data/meta_data.csv') # 콘텐츠 일반 메타
meta_plus = pd.read_csv('../data/meta_data_plus.csv') # 콘텐츠 확장 정보
profile = pd.read_csv('../data/profile_data.csv') # 프로필 정보
search = pd.read_csv('../data/search_data.csv') # 검색을 통한 시청 데이터
sample = pd.read_csv('../data/sample_submission.csv')


/var/folders/fb/fgwpc_kd3dxdcfsnm_9w3wv40000gn/T/ipykernel_68650/748092468.py:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv('../data/meta_data.csv') # 콘텐츠 일반 메타


In [16]:
his_group = history.groupby(['profile_id','album_id']).sum().reset_index()
his_group['watch']= 1
his_group

,profile_id,album_id,ss_id,log_time,payment,watch
0,3,15,20220301115653,20220301115719,0.0,1
1,3,16,20220301115653,20220301115809,0.0,1
2,3,17,20220301115653,20220301115958,0.0,1
3,3,18,20220301115653,20220301120118,0.0,1
4,3,19,20220301115653,20220301120229,0.0,1
...,...,...,...,...,...,...
380542,33032,7105,424628959523801,424628959623429,0.0,1
380543,33032,8910,20220426115550,20220426130410,0.0,1
380544,33032,14662,20220427112116,20220427112335,0.0,1
380545,33032,14663,40440854224232,40440854224753,0.0,1


In [19]:
customers = list(np.sort(his_group['profile_id'].unique()))
album = list(his_group['album_id'].unique())
watch = list(his_group['watch'])

rows = his_group['profile_id'].astype('category').cat.codes
cols = his_group['album_id'].astype('category').cat.codes
print(len(customers)) # 2663
print(len(album))  # 223
purchase_sparse = sparse.csr_matrix((watch, (rows, cols)), shape = (len(customers),len(album)))
purchase_sparse #2663 * 223 행렬

8311
20695


<8311x20695 sparse matrix of type '<class 'numpy.int64'>'
	with 380547 stored elements in Compressed Sparse Row format>

In [20]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

99.77874678528406

In [21]:
als_model = ALS(factors=20, regularization=0.01, iterations = 100)
als_model.fit(purchase_sparse)

  0%|          | 0/100 [00:00<?, ?it/s]

In [23]:
result = als_model.recommend(2, purchase_sparse[2], N=10)
result

(array([ 239, 1791,  342,  327, 1963,   36,    0,  357,  373,  186],
       dtype=int32),
 array([0.03730747, 0.03572573, 0.03559886, 0.02624757, 0.02400399,
        0.02161463, 0.02082878, 0.01873153, 0.01698042, 0.01322456],
       dtype=float32))

In [15]:
his_group

,profile_id,album_id,ss_id,log_time,payment
0,3,15,20220301115653,20220301115719,1
1,3,16,20220301115653,20220301115809,1
2,3,17,20220301115653,20220301115958,1
3,3,18,20220301115653,20220301120118,1
4,3,19,20220301115653,20220301120229,1
...,...,...,...,...,...
380542,33032,7105,424628959523801,424628959623429,1
380543,33032,8910,20220426115550,20220426130410,1
380544,33032,14662,20220427112116,20220427112335,1
380545,33032,14663,40440854224232,40440854224753,1
